# Investigate GLMsingle on control_beta subjects (spring 2022)
---------------------

##### Visualize output interactively.

For sanity checking.

### Import function libraries

In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import scipy.io as sio
import matplotlib.pyplot as plt
import nibabel as nib
from nilearn import plotting

import h5py
import os
from os.path import join, exists, split
import sys
import time
import urllib.request
import copy
import warnings
from tqdm import tqdm
from pprint import pprint
import sys
import datetime
import getpass
warnings.filterwarnings('ignore')

user = getpass.getuser()
print(f'Running as user {user}')
if user != 'gt':
    root = '/om5/group/evlab/u/gretatu/GLMsingle/'
else:
    root = '/Users/gt/om5/GLMsingle/'
    
os.chdir(root)

import glmsingle
from glmsingle.glmsingle import GLM_single

/opt/anaconda3/envs/glmdenoise/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


Running as user gt


 ---------- Running version from /Users/gt/om5/GLMsingle ---------- 




In [2]:
### RESOURCES ###
d_UID_to_session_list = {848: ['FED_20220420b_3T1', 'FED_20220427a_3T1'],
						 853: ['FED_20211008a_3T1', 'FED_20211013b_3T1'],
						 865: ['FED_20220414b_3T1', 'FED_20220415a_3T1'],
						 875: ['FED_20220408a_3T1', 'FED_20220411a_3T1'],
						 876: ['FED_20220413a_3T1', 'FED_20220420a_3T1']}

### Set paths and download the example dataset

In [3]:
# subject args
UID = 865
if len(str(UID)) < 3:
    UID_str = f'0{UID}'
else:
    UID_str = str(UID)

# GLM args
pcstop = -1
fracs = 0.975

# data args
preproc = 'swr'

# get path to the directory to which GLMsingle was installed
homedir = split(os.getcwd())[0]
htmldir = join(root, 'output_glmsingle', 'html_plots')
outputdir = join(root, 'output_glmsingle', f'output_glmsingle_preproc-{preproc}_pcstop{pcstop}_fracs-{fracs}_UID-{UID}')
datadir = '/mindhive/evlab/u/Shared/SUBJECTS'
headersession = f'{UID_str}_{d_UID_to_session_list[UID][0]}'
print(headersession)

865_FED_20220414b_3T1


In [4]:
os.listdir(outputdir)

['TYPEA_ONOFF.hdf5',
 'onoffR2.png',
 'meanvol.png',
 'onoffR2hist.png',
 'TYPEB_FITHRF.hdf5',
 'HRFindex.png',
 'TYPEC_FITHRF_GLMDENOISE.hdf5',
 'noisepool.png',
 'TYPED_FITHRF_GLMDENOISE_RR.hdf5',
 'typeD_R2.png',
 'FRACvalue.png']

In [5]:
typea = h5py.File(join(outputdir, 'TYPEA_ONOFF.hdf5'), 'r')
typeb = h5py.File(join(outputdir, 'TYPEB_FITHRF.hdf5'), 'r')
typec = h5py.File(join(outputdir, 'TYPEC_FITHRF_GLMDENOISE.hdf5'), 'r')
typed = h5py.File(join(outputdir, 'TYPED_FITHRF_GLMDENOISE_RR.hdf5'), 'r')

for name in typed:
    print(name)

FRACvalue
HRFindex
HRFindexrun
R2
R2run
betasmd
meanvol
noisepool
pcnum
pcregressors
rrbadness
scaleoffset


### Summary of important outputs

In [6]:
# the outputs of GLMsingle are formally documented in its
# header. here, we highlight a few of the more important outputs:

# R2 -> is model accuracy expressed in terms of R^2 (percentage).

# betasmd -> is the full set of single-trial beta weights (X x Y x Z x
# TRIALS). beta weights are arranged in chronological order.

# HRFindex -> is the 1-index of the best fit HRF. HRFs can be recovered
# with getcanonicalHRFlibrary(stimdur,tr)

# FRACvalue -> is the fractional ridge regression regularization level
# chosen for each voxel. values closer to 1 mean less regularization.

### Plot a slice of brain showing GLMsingle outputs

In [7]:
xyz = np.array(typed['R2']).shape

In [ ]:
# we are going to plot several outputs from the FIT_HRF_GLMdenoise_RR GLM,
# which contains the full set of GLMsingle optimizations.

# we will plot betas, R2, optimal HRF indices, and the voxel frac values
plot_fields = ['betasmd','R2','HRFindex',] # 'FRACvalue'
colormaps = ['RdBu_r','hot','jet','copper']
clims = [[-1,1],[0,85],[0,20],[0,1]]

plt.figure(figsize=(12,8))

for i in range(len(plot_fields)):
    print(i)
    
    plt.subplot(2,2,i+1)
    
    if plot_fields[i] == 'betasmd':
        # when plotting betas, for simplicity just average across all image presentations
        # this will yield a summary of whether voxels tend to increase or decrease their 
        # activity in response to the experimental stimuli (similar to outputs from 
        # an ONOFF GLM)
        plot_data = np.nanmean((np.array(typed[plot_fields[i]])),3)
        titlestr = 'average GLM betas'
    
    else:
        # plot all other voxel-wise metrics as outputted from GLMsingle
        plot_data = np.squeeze(np.array(typed[plot_fields[i]]).reshape(xyz))
        titlestr = plot_fields[i]
    
    plt.imshow(plot_data[:,:,65],cmap=colormaps[i],clim=clims[i]) # added indexing here, otherwise it errrors
    plt.colorbar()
    plt.title(titlestr)
    plt.axis(False)

0


### View more outputs from all the models

In [ ]:
type_str = 'typed'

if type_str == 'typec':
    type_of_interest = typec
elif type_str == 'typed':
    type_of_interest = typed
else:
    raise ValueError()

plot_fields = ['meanvol','betasmd','R2','HRFindex','noisepool',]
d_cmap = {'betasmd':'RdBu_r', 
         'R2':'hot',
         'HRFindex':'jet',
         'noisepool':'hot',
         'pcvoxels':'hot',
         'FRACvalue':'copper',
         'glmbadness':'hot',
         'meanvol':'Greys'}

d_clim = {'betasmd': [-1,1], 
         'R2': [30,90],
         'HRFindex':[0,20],
         'noisepool':[0,1],
         'pcvoxels':[0,1],
         'FRACvalue':[0,1],
         'glmbadness':[0,10],
         'meanvol':[0,800]}

In [ ]:
plt.figure(figsize=(20,14))

for i in range(len(plot_fields)):
    
    plt.subplot(2,4,i+1)
    
    if plot_fields[i] == 'betasmd' or plot_fields[i] == 'glmbadness':
        plot_data = np.nanmean((np.array(type_of_interest[plot_fields[i]])),3) # avg over trials
        titlestr = plot_fields[i]
    
    else:
        # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
        plot_data = np.squeeze(np.array(type_of_interest[plot_fields[i]]).reshape(xyz))
        titlestr = plot_fields[i]
    
    plt.imshow(plot_data[:,:,50],cmap=d_cmap[plot_fields[i]],clim=d_clim[plot_fields[i]])
    plt.colorbar()
    plt.title(f'{titlestr} - axial slice, (z=50)') # right is front here
    plt.axis(False)

In [ ]:
np.nanmean(np.squeeze(results_glmsingle['typec']['betasmd']),3).shape # glm betas

In [ ]:
np.max(results_glmsingle['typec']['meanvol'])

In [ ]:

def get_3D_coords(shape):
    """Get 3D coordinates for the flattened voxel array (column to coordinate)
    Based on these coords, it is possible to create an empty brain matrix of shape [shape] and fill in values.
    """
    indices = []
    for idx in np.ndindex(shape):
        indices.append(np.array(idx))
    # print('OBS: adding +1 for MATLAB')
    coords = (np.array(indices)).astype(int)
    return coords


def fill_empty_brain_matrix(shape, vals_of_interest, coords_3d, save_plot=False, save_nii=False,
                            save_str='', headerpath='/mindhive/evlab/u/Shared/SUBJECTS'):
    """
    Fill an 'empty' brain matrix with values to plot.

    :param shape: Tuple, the 3D shape of the brain space, e.g. (91, 109, 91)
    :param header_file: str, path to a .nii file in the same coordinate system
    :param vals_of_interest: the flattened "brain" array to plot, i.e. if shape is (91, 109, 91) this
    has size 902,629. Or in the original shape (3D). If so, only header info is added.
    :return:
    """
    if len(vals_of_interest.shape) == 1: # Flat array given
        ## Plot values of interest in a 3d brain matrix
        empty_brain = np.zeros([shape[0], shape[1], shape[2]])
        for idx, i in enumerate(vals_of_interest):
            x = coords_3d[idx][0]
            y = coords_3d[idx][1]
            z = coords_3d[idx][2]
            empty_brain[x, y, z] = i
    elif len(vals_of_interest.shape) == 3: # 3D brain already! just add header info.
        empty_brain = vals_of_interest
    else:
        raise ValueError('invalid shape!')
    
    # Load a header file
    n1_img = nib.load(headerpath)
    new_header = header = n1_img.header.copy()
    
    new_img = nib.nifti1.Nifti1Image(empty_brain, n1_img.affine, header=new_header)
    
    if save_plot:
        plotting.plot_stat_map(new_img, output_file=join(PLOTDIR, f'{UID}_{SESSION}_{FL}_{save_str}_statmap.png'))
        plotting.plot_glass_brain(new_img, output_file=join(PLOTDIR, f'{UID}_{SESSION}_{FL}_{save_str}_glassbrain.png'),
                                  colorbar=True, alpha=0.1)
        plotting.plot_roi(new_img, output_file=join(PLOTDIR, f'{UID}_{SESSION}_{FL}_{save_str}_roi.png'))
    
    if save_nii:
        nib.save(new_img, join(NEWNIIDIR, f'{UID_SESSION}_{FL}_{save_str}.nii'))
        
    return new_img

In [ ]:
## 3D coordinate info ##
coords_3d = get_3D_coords(shape=xyz)
vals_of_interest = np.squeeze(results_glmsingle['typec'][plot_fields[3]].flatten())
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=vals_of_interest,
                             coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=True)

## Loop across 3D images 

Save as html files!

In [ ]:
# Find header image that fits the UID and preproc of interest
# Load a header file
possible_header_imgs = os.listdir(join(datadir, headersession, 'nii')) # find any nii file that matches the input neural data
possible_header_imgs = [x for x in possible_header_imgs if x.startswith(preproc)]

PATH_header_img = join(datadir, headersession, 'nii', possible_header_imgs[0])
print(f'Using {PATH_header_img} as the header file\n')

In [ ]:
for i in range(len(plot_fields)):
    coords_3d = get_3D_coords(shape=xyz)

    if plot_fields[i] == 'betasmd':
        plot_data = np.nanmean((np.array(type_of_interest[plot_fields[i]])),3) # avg over trials
    
    else:
        # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
        plot_data = np.squeeze(np.array(type_of_interest[plot_fields[i]]))

    titlestr = plot_fields[i]
        
    # Feed to fill brain function
    img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                             coords_3d=coords_3d, headerpath=PATH_header_img)


    view = plotting.view_img(img, symmetric_cmap=True, 
                             vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                            title=titlestr, cmap=d_cmap[plot_fields[i]])
    
#     view.open_in_browser()
    view.save_as_html(join(htmldir, f'{titlestr}_preproc-{preproc}_pcstop{pcstop}_fracs-{fracs}_UID-{UID}.html'))

In [ ]:
plot_fields

In [ ]:
i=0
if plot_fields[i] == 'betasmd':
    plot_data = np.nanmean(np.squeeze(results_glmsingle['typec'][plot_fields[i]]),3) # avg over trials
    titlestr = 'average GLM betas (1000 stimuli)'

else:
    # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
    plot_data = np.squeeze(results_glmsingle['typec'][plot_fields[i]])
    titlestr = plot_fields[i]

# Feed to fill brain function
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                         coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=True, 
                         vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                        title=titlestr, cmap=d_cmap[plot_fields[i]])

In [ ]:
i=1
if plot_fields[i] == 'betasmd':
    plot_data = np.nanmean(np.squeeze(results_glmsingle['typec'][plot_fields[i]]),3) # avg over trials
    titlestr = 'average GLM betas (1000 stimuli)'

else:
    # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
    plot_data = np.squeeze(results_glmsingle['typec'][plot_fields[i]])
    titlestr = plot_fields[i]

# Feed to fill brain function
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                         coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=True, 
                         vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                        title=titlestr, cmap=d_cmap[plot_fields[i]])

In [ ]:
i=2
if plot_fields[i] == 'betasmd':
    plot_data = np.nanmean(np.squeeze(results_glmsingle['typec'][plot_fields[i]]),3) # avg over trials
    titlestr = 'average GLM betas (1000 stimuli)'

else:
    # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
    plot_data = np.squeeze(results_glmsingle['typec'][plot_fields[i]])
    titlestr = plot_fields[i]

# Feed to fill brain function
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                         coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=False, 
                         vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                        title=titlestr, cmap=d_cmap[plot_fields[i]])

In [ ]:
d_clim[plot_fields[i]][0]

In [ ]:
plotting.view_img(img, symmetric_cmap=False, 
                        title=titlestr, annotate=True)

In [ ]:
i=3
if plot_fields[i] == 'betasmd':
    plot_data = np.nanmean(np.squeeze(results_glmsingle['typec'][plot_fields[i]]),3) # avg over trials
    titlestr = 'average GLM betas (1000 stimuli)'

else:
    # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
    plot_data = np.squeeze(results_glmsingle['typec'][plot_fields[i]])
    titlestr = plot_fields[i]

# Feed to fill brain function
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                         coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=True, 
                         vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                        title=titlestr, cmap=d_cmap[plot_fields[i]])

In [ ]:
i=4
if plot_fields[i] == 'betasmd':
    plot_data = np.nanmean(np.squeeze(results_glmsingle['typec'][plot_fields[i]]),3) # avg over trials
    titlestr = 'average GLM betas (1000 stimuli)'

else:
    # plot all other voxel-wise metrics as outputted from GLMsingle and reshape to the brain
    plot_data = np.squeeze(results_glmsingle['typec'][plot_fields[i]])
    titlestr = plot_fields[i]

# Feed to fill brain function
img = fill_empty_brain_matrix(shape=xyz, vals_of_interest=plot_data,
                         coords_3d=coords_3d, )


plotting.view_img(img, symmetric_cmap=True, 
                         vmin=d_clim[plot_fields[i]][0], vmax=d_clim[plot_fields[i]][1],
                        title=titlestr, cmap=d_cmap[plot_fields[i]])

## Compare how the R2 value changes by fitting HRF for each voxel (type C) model vs simply one HRF for each voxel

In [ ]:
results_glmsingle.keys()

In [ ]:
results_glmsingle['typec'].keys()

In [ ]:
(results_glmsingle['typec']['R2']).shape

In [ ]:
np.nanmean(np.nanmean(results_glmsingle['typec']['R2']))

In [ ]:
np.nanmean(np.nanmean(results_glmsingle['typeb']['R2']))

In [ ]:
(results_glmsingle['typea']).keys()

In [ ]:
np.nanmean(np.nanmean(results_glmsingle['typea']['onoffR2']))

### Run a baseline GLM to compare with GLMsingle outputs

In [ ]:
# for comparison purposes we are going to run a standard GLM
# without HRF fitting, GLMdenoise, or ridge regression regularization. we
# will compute the split-half reliability at each voxel using this baseline
# GLM, and then assess whether reliability improves using the output betas
# from GLMsingle. 

# output directory for baseline GLM
outputdir_baseline = join(outputdir,'GLMbaseline')

# we will run this baseline GLM by changing the default settings in GLMsingle 
# contained within the "opt" structure.
opt = dict() 

# turn off optimizations 
opt['wantlibrary'] = 0 # switch off HRF fitting
opt['wantglmdenoise'] = 0 # switch off GLMdenoise
opt['wantfracridge'] = 0 # switch off ridge regression


# for the purpose of this example we will keep the relevant outputs in memory
# and also save them to the disk...
# the first two indices are the ON-OFF GLM and the baseline single-trial GLM. 
# no need to save the third (+ GLMdenoise) and fourth (+ fracridge) outputs
# since they will not even be computed
opt['wantmemoryoutputs'] = [1,1,0,0] 
opt['wantfileoutputs'] = [1,1,0,0]

# running python GLMsingle involves creating a GLM_single object
# and then running the procedure using the .fit() routine
glmbaseline_obj = GLM_single(opt)

# visualize the hyperparameters, including the modified baseline opts
pprint(glmbaseline_obj.params)

In [ ]:
start_time = time.time()

# if these outputs don't already exist, we will perform the call to
# GLMsingle; otherwise, we will just load from disk.
if not exists(outputdir_baseline):
    
    print(f'running GLMsingle...')

    # run GLMsingle, fitting the baseline GLM
    results_assumehrf = glmbaseline_obj.fit(
       design,
       data,
       stimdur,
       tr,
       outputdir=outputdir_baseline)
    
else:
    
    print(f'loading existing GLMsingle outputs from directory:\n\t{outputdir_glmsingle}')
    
    results_assumehrf = dict()
    results_assumehrf['typea'] = np.load(join(outputdir_baseline,'TYPEA_ONOFF.npy'),allow_pickle=True).item()
    results_assumehrf['typeb'] = np.load(join(outputdir_baseline,'TYPEB_FITHRF.npy'),allow_pickle=True).item()
    
    # note that even though we are loading TYPEB_FITHRF betas, HRF fitting
    # has been turned off and this struct field will thus contain the
    # outputs of a GLM fit using the canonical HRF.
    
elapsed_time = time.time() - start_time
print(
    '\telapsed time: ',
    f'{time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}'
)

In [ ]:
# create dictionary containing the GLM betas from the four different models we will compare.
# note that the "assume hrf" betas come from the "typeb" field of our baseline GLM
# (with HRF fitting turned off), and that the "fit hrf" betas also come from 
# the "typeb" field of the GLM that ran with all default GLMsingle routines
# enabled

models = dict()
models['assumehrf'] = results_assumehrf['typeb']['betasmd'].reshape(xyz + (750,))
models['fithrf'] = results_glmsingle['typeb']['betasmd']
models['fithrf_glmdenoise'] = results_glmsingle['typec']['betasmd']
models['fithrf_glmdenoise_rr'] = results_glmsingle['typed']['betasmd']

### Visualize cortical ROI defining visually-responsive areas

In [ ]:
# get mask defining liberal visual cortex ROI. "nsdgeneral" is a general ROI 
# that was manually drawn on fsaverage covering voxels responsive to the NSD experiment 
# in the posterior aspect of cortex. for the sake of simplicity we will focus 
# on voxels within this ROI in computing split-half reliability

nsdgeneral_roi = roi.astype(float)

# convert voxels outside ROI to nan for overlay plotting
nsdgeneral_roi[nsdgeneral_roi==0] = np.nan 

# get mean fMRI volume from run 1
meanvol = np.squeeze(np.mean(data[0].reshape(xyzt),axis=3))

# plot ROI on top of overlay
plt.figure(figsize=(12,6))
plt.imshow(meanvol,cmap='gray')
plt.imshow(nsdgeneral_roi,cmap='Blues',clim=(0,2))

plt.title('voxels in nsdgeneral ROI')
plt.box(False)
plt.axis(False);

### Compute median split-half reliability within the ROI for each beta version

In [ ]:
# finally, let's compute split-half reliability. we are going to loop
# through our 4 models and calculate split-half reliability for each of them

vox_reliabilities = [] # output variable for reliability values

modelnames = list(models.keys())

# for each beta version...
for m in range(len(modelnames)):
    
    print(f'computing reliability for beta version: {modelnames[m]}')
    time.sleep(1)
    
    # get the repeated-condition GLM betas using our repindices variable
    betas = models[modelnames[m]][:,:,:,repindices] # automatically reshapes to (X x Y x Z x 2 x nConditions)
    x,y,z = betas.shape[:3] 
    
    rels = np.full((x,y,z),np.nan)
    
    # loop through voxels in the 3D volume...
    for xx in tqdm(range(x)):
        for yy in range(y):
            for zz in range(z):
                
                # reliability at a given voxel is pearson correlation between response profiles from first and 
                # second image presentations (dim = 136 conditions)
                rels[xx,yy,zz] = np.corrcoef(betas[xx,yy,zz,0],
                                             betas[xx,yy,zz,1])[1,0]
          
    vox_reliabilities.append(rels)


### Assess change in reliability yielded by GLMsingle

In [ ]:
# for each GLM we will calculate median reliability for voxels within the
# nsdgeneral visual ROI and compare using a bar graph

comparison = []
for vr in vox_reliabilities:
    comparison.append(np.nanmedian(vr[nsdgeneral_roi==1]))

plt.figure(figsize=(18,6))
plt.subplot(121)
plt.bar(np.arange(len(comparison)),comparison,width=0.5)
plt.title('Median voxel split-half reliability of GLM models')
plt.xticks(np.arange(4),np.array(['ASSUMEHRF', 'FITHRF', 'FITHRF\nGLMDENOISE', 'FITHRF\nGLMDENOISE\nRR']));
plt.ylim([0.1,0.2])

# draw plot showing the change in reliability between the baseline GLM
# and the final output of GLMsingle (fithrf-glmdenoise-RR betas)
vox_improvement = np.squeeze(vox_reliabilities[3] - vox_reliabilities[0])
vox_improvement[nsdgeneral_roi != 1] = np.nan

plt.subplot(122)
plt.imshow(meanvol,cmap='gray',aspect='auto')
plt.imshow(vox_improvement,cmap='RdBu_r',clim=(-0.3,0.3),aspect='auto')
plt.colorbar()
plt.title('change in nsdgeneral voxel reliability**\ndue to GLMsingle (r)')
plt.xticks([])
plt.yticks([])
plt.xlabel('\n**plotting (FITHRF_GLMDENOISE_RR - ASSUMEHRF) reliabilities');

# notice that there is systematic increase in reliability moving from the
# first to the second to the third to the final fourth version of the GLM
# results. these increases reflect, respectively, the addition of HRF
# fitting, the derivation and use of data-driven nuisance regressors, and
# the use of ridge regression as a way to regularize the instability of
# closely spaced experimental trials. depending on one's experimental
# goals, it is possible with setting of option flags to activate a subset
# of these analysis features.

# also, keep in mind that in the above figure, we are simply showing the
# median as a metric of the central tendency (you may want to peruse
# individual voxels in scatter plots, for example).